In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import requests
import mpu
import os
import json

In [8]:
dataset=pd.read_csv("./delivery_data_pulito.csv")#lines=true perchè ogni riga del file.json è un json, devo creare tabella

#dataset.info()

dataset.describe()
#dataset['VEHICLETYPE'].unique()

,Unnamed: 0,DEPARTURE_ZIPCODE,ARRIVAL_ZIPCODE,GROSS_WEIGHT_KG,NET_WEIGHT_KG,VOLUME_M3,DECLARED_DISTANCE_KM,DELIVERY_TIME_HH,WDAY
count,14554.000000,14548.000000,14554.000000,14554.000000,14554.000000,14554.000000,14554.000000,14554.000000,14554.000000
mean,7277.500000,53013.321694,41699.581352,9.744264,8.002172,0.188305,925.313013,81.338395,1.730177
std,4201.522244,22512.124864,30222.758997,34.946037,31.718243,0.983377,1320.171381,73.386400,1.507748
min,1.000000,12.000000,10.000000,0.020000,0.000000,0.000000,0.640000,0.000000,0.000000
25%,3639.250000,56122.750000,17031.000000,1.000000,0.000000,0.010000,240.940000,38.000000,0.000000
50%,7277.500000,63076.000000,35042.000000,1.370000,0.000000,0.010000,428.230000,72.000000,1.000000
75%,10915.750000,63076.000000,66020.000000,3.160000,2.280000,0.110000,547.950000,108.000000,3.000000
max,14554.000000,99208.000000,99518.000000,1039.820000,948.840000,105.000000,11818.310000,1920.000000,6.000000


In [ ]:
# Leggi il file CSV originale
df = pd.read_csv('delivery_data.csv')

# Conta le righe con NaN prima di rimuoverle
nan_count = df['SERVICETYPE'].isna().sum()

# Rimuovi le righe con NaN in SERVICETYPE
df = df.dropna(subset=['SERVICETYPE'])

# Conta le spedizioni IT-US prima di rimuoverle
it_us_count = (
    ((df['DEPARTURE_COUNTRY'] == 'IT') & (df['ARRIVAL_COUNTRY'] == 'US')) | 
    ((df['DEPARTURE_COUNTRY'] == 'US') & (df['ARRIVAL_COUNTRY'] == 'IT'))
).sum()

# Rimuovi le spedizioni tra IT e US in entrambe le direzioni
mask_it_us = ~(
    ((df['DEPARTURE_COUNTRY'] == 'IT') & (df['ARRIVAL_COUNTRY'] == 'US')) | 
    ((df['DEPARTURE_COUNTRY'] == 'US') & (df['ARRIVAL_COUNTRY'] == 'IT'))
)
df = df[mask_it_us]

# Crea i tre dataframe separati basati su SERVICETYPE
# 1. Corrieri
corrieri_df = df[df['SERVICETYPE'].isin(['Corriere espresso', 'Corriere espresso resi'])]

# 2. Via gomma
gomma_df = df[df['SERVICETYPE'] == 'Via gomma']

# 3. E-commerce
ecommerce_df = df[df['SERVICETYPE'].isin(['E-commerce', 'E-commerce Resi'])]

# Salva i dataframe in file CSV separati
corrieri_df.to_csv('corrieri_data.csv', index=False)
gomma_df.to_csv('gomma_data.csv', index=False)
ecommerce_df.to_csv('ecommerce_data.csv', index=False)

# Stampa informazioni dettagliate
print("Statistiche dei file generati:")
print(f"\nRighe nel file corrieri: {len(corrieri_df)}")
print(f"Righe nel file gomma: {len(gomma_df)}")
print(f"Righe nel file ecommerce: {len(ecommerce_df)}")

# Stampa le righe rimosse
print(f"\nRighe rimosse con NaN in SERVICETYPE: {nan_count}")
print(f"Righe rimosse per spedizioni IT-US: {it_us_count}")

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ## Getting a zip distance matrix for easy lookup

# Necessary Imports

import os
import pandas as pd
import numpy as np
import mpu
import json

import warnings
warnings.filterwarnings("ignore")

# Helper functions
def walk_up_folder(path, depth=1):
    """
    Helper method to navigate the file system and get to the file location
    """
    _cur_depth = 1
    while _cur_depth < depth:
        path = os.path.dirname(path)
        _cur_depth += 1
    return path

def geodist(coord1,coord2):
    """
    Calculate the distance between
    (lat1, lon1), (lat2, lon2)
    """
    # Convert to miles 1km = 0.621371 miles
    return round(mpu.haversine_distance(coord1,coord2)*0.621371,2)

def fill_lfromu(A):
    """
    Fill out the symmetric values in the lower triangle based on upper
    Not needed since it is covered in the for loop above
    """
    out = A.T + A
    np.fill_diagonal(out,0)
    return out

import numba as nb
#@nb.njit(parallel=True,fastmath=True)
def func(len_matrix,coordinates):
    zip_dist = np.zeros((len_matrix, ) * 2)
    indices = np.arange(len_matrix)
    # Get upper triangular pairs
    fill_cells = np.stack(np.triu_indices(len_matrix), axis=1)
    # Loop through upper triangular indices while avoiding diagonal element indices
    for idx in fill_cells:
        i,j = indices[idx]
        if i!=j:
            zip_dist[i][j]= zip_dist[j][i] = geodist(coordinates[i],coordinates[j])
    return zip_dist

# ## Loading raw data
# 
# Source - https://github.com/symerio/postal-codes-data/blob/master/data/geonames/US.txt 
# 
# **Geocoding format**
# The result of a geo-localistion query is a pandas.DataFrame with the following columns,
# 
# country_code: iso country code, 2 characters  
# postal_code : postal code  
# place_name : place name (e.g. town, city etc)  
# state_name : 1. order subdivision (state)  
# state_code : 1. order subdivision (state)  
# county_name : 2. order subdivision (county/province)  
# county_code : 2. order subdivision (county/province)  
# community_name : 3. order subdivision (community)  
# community_code : 3. order subdivision (community)  
# latitude : estimated latitude (wgs84)  
# longitude : estimated longitude (wgs84)  
# accuracy : accuracy of lat/lng from 1=estimated to 6=centroid  


if __name__ =="__main__":
    
    data_dir = os.path.join(walk_up_folder(os.getcwd(), 2), "Data/")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("Data folder created")
        
    # Load USZips data
    source_file = os.path.join(data_dir,"US_zips.txt")
    if os.path.exists(source_file):
        print("Loading the source file")
        print("Processing the dataframe")
        zip_df = pd.read_csv(source_file),sep='\t',header=None,
                             names=["country_code","postal_code","place_name","state_name","state_code",
                                    "county_name","county_code","community_name","community_code",
                                    "latitude","longitude","accuracy"])

        # Dropping unnecessary columns
        zip_df.drop(["country_code","place_name","state_name","county_name","county_code","community_name","community_code"],axis=1,inplace=True)
        # There were 2 duplicate instances
        zip_df.drop_duplicates(subset=["postal_code"],keep='last',inplace=True)
        # Coordinate column as (lat,long)
        zip_df["coord"]=list(zip(zip_df["latitude"],zip_df["longitude"]))
        zip_df["mapping"] = zip_df.index # Column with index

        # ## Lookup Matrix
        #
        # The idea is to form a (41483,41483) matrix where the cell values are the haversize distances between two zips that are indexed.
        # I'll save the index look up and the matrix file as json or pkl


        # Dictionary mapping the zipcode to index
        zip_index = dict(zip(zip_df["postal_code"].astype(str),zip_df["mapping"]))
        print(len(zip_index))
        # Saving the indexer
        with open(os.path.join(data_dir, "zips_indexer.json")), 'w') as f:
            json.dump(zip_index, f)
        print("ZiptoIndex json saved!")
        
        # Matrix
        len_matrix = zip_df.shape[0]
        coordinates = zip_df.coord.values
        
        print("Creating the pairwise distance matrix! This might take an hour, why don't you go get something to eat? ")
        # Takes 1hour. Unfortunately numba.jit doesn't work with user defined functions ")
        zip_dist = func(len_matrix,coordinates)
        # Convert to float32
        print("ZipDist Matrix saving!")
        np.savez_compressed(os.path.join(data_dir, "zip_dist.npz")), np.float32(zip_dist))
    else:
        print("Please ensure you have the source file of the raw zips data in the Data directory and try again !")

In [3]:
import pandas as pd
import requests
from geopy.geocoders import Nominatim
from typing import Optional, Tuple
from tqdm import tqdm

# Carica il dataset
dataSet = pd.read_csv("validated_delivery_data.csv")

def get_osrm_distance(coord1, coord2):
    url = f"http://router.project-osrm.org/route/v1/driving/{coord1[1]},{coord1[0]};{coord2[1]},{coord2[0]}"
    
    response = requests.get(url)
    data = response.json()
    
    if 'routes' in data and len(data['routes']) > 0:
        distance_km = data['routes'][0]['distance'] / 1000
        duration_seconds = data['routes'][0]['duration']
        return round(distance_km, 2), duration_seconds
    return None, None

def get_zipcode_coordinates(zipcode: str, country) -> Optional[Tuple[float, float]]:
    try:
        geolocator = Nominatim(user_agent="my_distance_calculator")
        location = geolocator.geocode(f"{zipcode}, {country}")
        
        if location:
            return (location.latitude, location.longitude)
        return None
        
    except Exception:
        return None

def print_statistics(processed_rows, successful_lookups, failed_lookups):
    total = successful_lookups + failed_lookups
    success_rate = (successful_lookups / total * 100) if total > 0 else 0
    failure_rate = (failed_lookups / total * 100) if total > 0 else 0
    
    print(f"\n--- Statistiche dopo {processed_rows} righe processate ---")
    print(f"Ricerche riuscite: {successful_lookups} ({success_rate:.2f}%)")
    print(f"Ricerche fallite: {failed_lookups} ({failure_rate:.2f}%)")
    print("------------------------------------------------\n")

# Filtra il DataFrame per le righe non ancora calcolate
filtered_data = dataSet[
    (dataSet['DECLARED_DISTANCE_KM'].isna() | (dataSet['DECLARED_DISTANCE_KM'] == 0))
]

# Contatori per le statistiche
processed_rows = 0
successful_lookups = 0
failed_lookups = 0

# Itera attraverso il dataset filtrato
for index, row in tqdm(filtered_data.iterrows(), total=filtered_data.shape[0]):
    processed_rows += 1
    
    departure_country = row['DEPARTURE_COUNTRY']
    departure_zipcode = row['DEPARTURE_ZIPCODE']
    arrival_zipcode = row['ARRIVAL_ZIPCODE']
    
    if departure_country in ["IT", "US"]:
        coordinataPartenza = get_zipcode_coordinates(departure_zipcode, departure_country)
        coordinataArrivo = get_zipcode_coordinates(arrival_zipcode, departure_country)
    else:
        failed_lookups += 1
        continue
    
    if coordinataPartenza and coordinataArrivo:
        distanza = None
        error_count = 0
        
        while error_count < 5:
            distanza, _ = get_osrm_distance(coordinataPartenza, coordinataArrivo)
            if distanza is not None:
                # Aggiorna il DataFrame e salva immediatamente
                dataSet.at[index, 'DECLARED_DISTANCE_KM'] = distanza
                dataSet.to_csv("validated_delivery_data.csv", index=False)
                
                successful_lookups += 1
                break
            error_count += 1
            
            if error_count == 5:
                print(f"Impossibile trovare la distanza id {index}")
                failed_lookups += 1
    else:
        failed_lookups += 1
    
    # Stampa le statistiche ogni 100 righe
    if processed_rows % 100 == 0:
        print_statistics(processed_rows, successful_lookups, failed_lookups)

# Stampa le statistiche finali
print_statistics(processed_rows, successful_lookups, failed_lookups)

print("Elaborazione completata. Dati salvati in validated_delivery_data.csv")

 17%|█▋        | 100/574 [03:27<15:53,  2.01s/it]


--- Statistiche dopo 100 righe processate ---
Ricerche riuscite: 100 (100.00%)
Ricerche fallite: 0 (0.00%)
------------------------------------------------



 35%|███▍      | 200/574 [06:52<12:26,  1.99s/it]


--- Statistiche dopo 200 righe processate ---
Ricerche riuscite: 200 (100.00%)
Ricerche fallite: 0 (0.00%)
------------------------------------------------



 52%|█████▏    | 300/574 [10:13<09:09,  2.00s/it]

...


--- Statistiche dopo 400 righe processate ---
Ricerche riuscite: 400 (100.00%)
Ricerche fallite: 0 (0.00%)
------------------------------------------------



 87%|████████▋ | 500/574 [16:58<02:29,  2.02s/it]


--- Statistiche dopo 500 righe processate ---
Ricerche riuscite: 500 (100.00%)
Ricerche fallite: 0 (0.00%)
------------------------------------------------



100%|██████████| 574/574 [19:32<00:00,  2.04s/it]


--- Statistiche dopo 574 righe processate ---
Ricerche riuscite: 574 (100.00%)
Ricerche fallite: 0 (0.00%)
------------------------------------------------

Elaborazione completata. Dati salvati in validated_delivery_data.csv


In [2]:
#creazione dataset uniti con orari giusti

import pandas as pd
import numpy as np

def clean_delivery_data(main_file, additional_files):
    """
    Procedura di pulizia e integrazione dei dati di consegna
    
    Args:
    main_file (str): Percorso del file CSV principale
    additional_files (list): Lista di percorsi dei file CSV aggiuntivi
    
    Returns:
    tuple: DataFrame con chilometri integrati e DataFrame con chilometri e tempi sistemati
    """
    # 1. Carica il dataset principale
    df = pd.read_csv(main_file)
    
    # 2. Crea una copia del dataset
    df_km_integrated = df.copy()
    
    # 3. Cerca di riempire i chilometri mancanti dagli altri dataset
    for additional_file in additional_files:
        additional_df = pd.read_csv(additional_file)
        
        # Trova le righe con chilometri mancanti
        mask_missing_km = df_km_integrated['DECLARED_DISTANCE_KM'].isna()
        
        # Cerca di integrare i chilometri dall'altro dataset
        for index, row in df_km_integrated[mask_missing_km].iterrows():
            match = additional_df[
                (additional_df['DEPARTURE_COUNTRY'] == row['DEPARTURE_COUNTRY']) & 
                (additional_df['ARRIVAL_COUNTRY'] == row['ARRIVAL_COUNTRY'])
            ]
            
            if not match.empty:
                df_km_integrated.at[index, 'DECLARED_DISTANCE_KM'] = match['DECLARED_DISTANCE_KM'].mode().iloc[0]
    
    # 4. Crea un ulteriore copia per gestire i tempi di viaggio negativi
    df_validated = df_km_integrated.copy()
    
    # 5. Gestisci tempi di viaggio negativi
    negative_travel_time = df_validated['DELIVERY_TIME_HH'] < 0
    
    # Converti tempi negativi in positivi e scambia le date
    df_validated.loc[negative_travel_time, 'DELIVERY_TIME_HH'] = abs(df_validated.loc[negative_travel_time, 'DELIVERY_TIME_HH'])
    df_validated.loc[negative_travel_time, ['SHIPPING_DATE', 'ACTUAL_DELIVERY_DATE']] = \
        df_validated.loc[negative_travel_time, ['ACTUAL_DELIVERY_DATE', 'SHIPPING_DATE']].values
    
    return df_km_integrated, df_validated

# Esempio di utilizzo
main_file = 'delivery_data.csv'
additional_files = ['gomma_data.csv', 'ecommerce_data.csv', 'corrieri_data.csv']

# Esegui la pulizia dei dati
km_integrated_data, validated_data = clean_delivery_data(main_file, additional_files)

# Salva i due dataset
km_integrated_data.to_csv('km_integrated_delivery_data.csv', index=False)
validated_data.to_csv('validated_delivery_data.csv', index=False)

print("Pulizia dei dati completata.")
print("1. Dataset con chilometri integrati salvato come 'km_integrated_delivery_data.csv'")
print("2. Dataset con chilometri e tempi sistemati salvato come 'validated_delivery_data.csv'")

Pulizia dei dati completata.
1. Dataset con chilometri integrati salvato come 'km_integrated_delivery_data.csv'
2. Dataset con chilometri e tempi sistemati salvato come 'validated_delivery_data.csv'


In [14]:
#sistemazione tipi di veicoli e servizzi definitivo

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
import warnings

# Sopprime gli specifici warning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        self.encoders = {}

    def fit(self, X, y=None):
        # Ensure X is a DataFrame
        X = pd.DataFrame(X)
        
        # If no columns specified, use all columns
        if self.columns is None:
            self.columns = X.columns

        # Fit label encoders for specified columns
        for col in self.columns:
            le = LabelEncoder()
            le.fit(X[col].astype(str).fillna('Unknown'))
            self.encoders[col] = le
        
        return self

    def transform(self, X):
        # Ensure X is a DataFrame
        X = pd.DataFrame(X)
        X_copy = X.copy()
        
        for col in self.columns:
            # Check if column exists and has an encoder
            if col in X_copy.columns and col in self.encoders:
                le = self.encoders[col]
                
                # Transform column, handling unseen and NaN values
                X_copy[col] = X_copy[col].fillna('Unknown')
                X_copy[col] = X_copy[col].map(
                    lambda s: le.transform([str(s)])[0] 
                    if str(s) in le.classes_ 
                    else -1
                )
        
        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)

def prepare_data_for_ml(df):
    """
    Prepara i dati per l'addestramento del modello di Machine Learning
    
    Args:
    df (pd.DataFrame): Dataset originale
    
    Returns:
    tuple: Features, target per SERVICETYPE e VEHICLETYPE
    """
    # Seleziona le feature per la predizione
    features_columns = [
        'DEPARTURE_COUNTRY', 
        'ARRIVAL_COUNTRY', 
        'GROSS_WEIGHT_KG', 
        'NET_WEIGHT_KG', 
        'VOLUME_M3', 
        'DECLARED_DISTANCE_KM',
        'DELIVERY_TIME_HH',
        'WDAY'
    ]
    
    # Crea una copia del dataset
    df_prepared = df.copy()
    
    # Rimuovi le righe con valori target nulli
    df_known = df_prepared.dropna(subset=['SERVICETYPE', 'VEHICLETYPE'])
    
    # Prepara le features
    X = df_known[features_columns].copy()
    
    # Prepara i target
    y_service = df_known['SERVICETYPE']
    y_vehicle = df_known['VEHICLETYPE']
    
    return X, y_service, y_vehicle

def create_ml_pipeline(is_categorical=True):
    """
    Crea una pipeline di preprocessing e classificazione
    
    Args:
    is_categorical (bool): Se True, usa encoding per le colonne categoriche
    
    Returns:
    Pipeline: Pipeline di preprocessing e classificazione
    """
    # Colonne numeriche e categoriche
    numeric_features = [
        'GROSS_WEIGHT_KG', 
        'NET_WEIGHT_KG', 
        'VOLUME_M3', 
        'DECLARED_DISTANCE_KM',
        'DELIVERY_TIME_HH',
        'WDAY'
    ]
    categorical_features = ['DEPARTURE_COUNTRY', 'ARRIVAL_COUNTRY']
    
    # Preprocessing per colonne numeriche
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # Preprocessing per colonne categoriche
    preprocessing_steps = [
        ('imputer', SimpleImputer(strategy='most_frequent'))
    ]
    
    # Aggiungi encoding se richiesto
    if is_categorical:
        preprocessing_steps.append(('encoder', MultiColumnLabelEncoder()))
    
    categorical_transformer = Pipeline(steps=preprocessing_steps)
    
    # Combina i preprocessori
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])
    
    # Pipeline completa con preprocessore e classificatore
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])
    
    return pipeline

def impute_with_ml(input_file):
    """
    Imputa i valori mancanti utilizzando Machine Learning
    
    Args:
    input_file (str): Percorso del file CSV di input
    
    Returns:
    pd.DataFrame: DataFrame con valori imputati
    """
    # Carica il dataset
    df = pd.read_csv(input_file)
    
    # Crea un duplicato del dataset
    df_imputed = df.copy()
    
    # Prepara i dati per l'addestramento
    X, y_service, y_vehicle = prepare_data_for_ml(df)
    
    # Dividi i dati in training e test
    X_train, X_test, y_service_train, y_service_test, y_vehicle_train, y_vehicle_test = train_test_split(
        X, y_service, y_vehicle, test_size=0.2, random_state=42
    )
    
    # Crea e addestra i modelli
    service_pipeline = create_ml_pipeline(is_categorical=True)
    vehicle_pipeline = create_ml_pipeline(is_categorical=True)
    
    # Addestra entrambi i modelli
    service_pipeline.fit(X_train, y_service_train)
    vehicle_pipeline.fit(X_train, y_vehicle_train)
    
    # Valuta i modelli
    service_pred = service_pipeline.predict(X_test)
    vehicle_pred = vehicle_pipeline.predict(X_test)
    
    # Calcola l'accuratezza
    service_accuracy = accuracy_score(y_service_test, service_pred) * 100
    vehicle_accuracy = accuracy_score(y_vehicle_test, vehicle_pred) * 100
    
    print(f"\nAccuratezza SERVICETYPE: {service_accuracy:.2f}%")
    print(f"Accuratezza VEHICLETYPE: {vehicle_accuracy:.2f}%")
    
    # Identifica le righe con valori mancanti
    missing_mask = df_imputed['SERVICETYPE'].isna() | df_imputed['VEHICLETYPE'].isna()
    missing_data = df_imputed.loc[missing_mask, X.columns].copy()
    
    # Imputa i valori mancanti
    for index in missing_mask[missing_mask].index:
        # Imputa SERVICETYPE
        if pd.isna(df_imputed.at[index, 'SERVICETYPE']):
            df_imputed.at[index, 'SERVICETYPE'] = service_pipeline.predict(missing_data.loc[[index]])[0]
        
        # Imputa VEHICLETYPE
        if pd.isna(df_imputed.at[index, 'VEHICLETYPE']):
            df_imputed.at[index, 'VEHICLETYPE'] = vehicle_pipeline.predict(missing_data.loc[[index]])[0]
    
    return df_imputed

# Esegui l'imputazione
input_file = 'validated_delivery_data.csv'
df_with_imputed_data = impute_with_ml(input_file)

# Salva il dataset con i valori imputati
df_with_imputed_data.to_csv('validated_delivery_data_ml_imputed.csv', index=False)

print("\nFile 'validated_delivery_data_ml_imputed.csv' creato con successo.")


Accuratezza SERVICETYPE: 96.72%
Accuratezza VEHICLETYPE: 90.55%

File 'validated_delivery_data_ml_imputed.csv' creato con successo.


In [8]:
#previsione non definitiva

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from datetime import datetime, timedelta

class DeliveryPredictor:
    def __init__(self):
        self.le_service = LabelEncoder()
        self.le_vehicle = LabelEncoder()
        self.le_dep_country = LabelEncoder()
        self.le_arr_country = LabelEncoder()
        self.le_dep_zip = LabelEncoder()
        self.le_arr_zip = LabelEncoder()
        self.scaler = RobustScaler()  # Cambiato a RobustScaler per gestire meglio gli outlier
        self.distance_model = GradientBoostingRegressor(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        self.time_model = GradientBoostingRegressor(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        
    def prepare_data(self, df):
        # Pulisci e prepara i dati
        df = df.copy()
        
        # Converti i CAP in stringhe e gestisci i valori NaN
        df['DEPARTURE_ZIPCODE'] = df['DEPARTURE_ZIPCODE'].fillna('00000')
        df['ARRIVAL_ZIPCODE'] = df['ARRIVAL_ZIPCODE'].fillna('00000')
        df['DEPARTURE_ZIPCODE'] = df['DEPARTURE_ZIPCODE'].astype(str).str.replace('.0', '').str.zfill(5)
        df['ARRIVAL_ZIPCODE'] = df['ARRIVAL_ZIPCODE'].astype(str).str.replace('.0', '').str.zfill(5)
        
        # Rimuovi outlier dal tempo di consegna
        q1 = df['DELIVERY_TIME_HH'].quantile(0.25)
        q3 = df['DELIVERY_TIME_HH'].quantile(0.75)
        iqr = q3 - q1
        df = df[
            (df['DELIVERY_TIME_HH'] >= q1 - 1.5 * iqr) & 
            (df['DELIVERY_TIME_HH'] <= q3 + 1.5 * iqr)
        ]
        
        # Aggiungi feature derivate
        df['SAME_COUNTRY'] = (df['DEPARTURE_COUNTRY'] == df['ARRIVAL_COUNTRY']).astype(int)
        
        # Encode categorical variables
        df['SERVICETYPE_encoded'] = self.le_service.fit_transform(df['SERVICETYPE'])
        df['VEHICLETYPE_encoded'] = self.le_vehicle.fit_transform(df['VEHICLETYPE'])
        df['DEPARTURE_COUNTRY_encoded'] = self.le_dep_country.fit_transform(df['DEPARTURE_COUNTRY'])
        df['ARRIVAL_COUNTRY_encoded'] = self.le_arr_country.fit_transform(df['ARRIVAL_COUNTRY'])
        df['DEPARTURE_ZIPCODE_encoded'] = self.le_dep_zip.fit_transform(df['DEPARTURE_ZIPCODE'])
        df['ARRIVAL_ZIPCODE_encoded'] = self.le_arr_zip.fit_transform(df['ARRIVAL_ZIPCODE'])
        
        # Store unique values
        self.unique_values = {
            'service_types': df['SERVICETYPE'].unique(),
            'vehicle_types': df['VEHICLETYPE'].unique(),
            'departure_countries': df['DEPARTURE_COUNTRY'].unique(),
            'arrival_countries': df['ARRIVAL_COUNTRY'].unique(),
            'departure_zipcodes': df['DEPARTURE_ZIPCODE'].unique(),
            'arrival_zipcodes': df['ARRIVAL_ZIPCODE'].unique()
        }
        
        # Prepare features
        features = [
            'SERVICETYPE_encoded', 'VEHICLETYPE_encoded', 
            'DEPARTURE_COUNTRY_encoded', 'ARRIVAL_COUNTRY_encoded',
            'DEPARTURE_ZIPCODE_encoded', 'ARRIVAL_ZIPCODE_encoded', 
            'WDAY', 'SAME_COUNTRY'
        ]
        
        # Aggiungi DISTANCE_ENCODED anche se non è presente nel dataset
        if 'DECLARED_DISTANCE_KM' in df.columns:
            df['DISTANCE_ENCODED'] = self.scaler.fit_transform(df[['DECLARED_DISTANCE_KM']])
        else:
            df['DISTANCE_ENCODED'] = 0  # Imposta a 0 se non è presente
        
        features.append('DISTANCE_ENCODED')
        
        X = df[features].values
        X = self.scaler.fit_transform(X)
        
        return X, df

    def train(self, data_path):
        # Load data
        df = pd.read_csv(data_path)
        
        # Prepare features and targets
        X, df_cleaned = self.prepare_data(df)
        y_distance = df_cleaned['DECLARED_DISTANCE_KM'].values
        y_time = df_cleaned['DELIVERY_TIME_HH'].values
        
        # Normalizza il tempo di consegna per ore lavorative (8 ore al giorno)
        y_time = np.minimum(y_time, 72)  # Cap a 72 ore (9 giorni lavorativi)
        
        # Split data
        X_train, X_test, y_dist_train, y_dist_test, y_time_train, y_time_test = train_test_split(
            X, y_distance, y_time, test_size=0.2, random_state=42
        )
        
        # Train models
        self.distance_model.fit(X_train, y_dist_train)
        self.time_model.fit(X_train, y_time_train)
        
        # Calculate accuracy
        dist_accuracy = r2_score(y_dist_test, self.distance_model.predict(X_test))
        time_accuracy = r2_score(y_time_test, self.time_model.predict(X_test))
        
        return dist_accuracy, time_accuracy
    
    def predict(self, service_type, vehicle_type, departure_country, arrival_country,
                departure_zipcode, arrival_zipcode, departure_date):
        # Convert departure_date to weekday (0 = Monday)
        departure_dt = datetime.strptime(departure_date, '%Y-%m-%d')
        wday = departure_dt.weekday()
        
        # Convert zipcodes to strings with leading zeros
        departure_zipcode = str(departure_zipcode).replace('.0', '').zfill(5)
        arrival_zipcode = str(arrival_zipcode).replace('.0', '').zfill(5)
        
        # Calculate same country feature
        same_country = 1 if departure_country == arrival_country else 0
        
        # Prepare input data
        input_data = np.array([[
            self.le_service.transform([service_type])[0],
            self.le_vehicle.transform([vehicle_type])[0],
            self.le_dep_country.transform([departure_country])[0],
            self.le_arr_country.transform([arrival_country])[0],
            self.le_dep_zip.transform([departure_zipcode])[0],
            self.le_arr_zip.transform([arrival_zipcode])[0],
            wday,
            same_country,
            0  # Aggiungi un valore di default per DISTANCE_ENCODED
        ]])
        
        input_scaled = self.scaler.transform(input_data)
        
        # Make predictions
        predicted_distance = self.distance_model.predict(input_scaled)[0]
        predicted_hours = self.time_model.predict(input_scaled)[0]
        
        # Limita le ore previste a un massimo ragionevole
        predicted_hours = min(predicted_hours, 72)
        
        # Calcola il numero di giorni di lavoro e le ore di lavoro
        work_hours_per_day = 8
        total_workdays = int(np.ceil(predicted_hours / work_hours_per_day))
        total_work_hours = total_workdays * work_hours_per_day
        
        # Calculate arrival date considering 8-hour workdays and weekends
        arrival_date = departure_dt
        
        for _ in range(total_workdays):
            arrival_date += timedelta(days=1)
            # Skip weekends
            while arrival_date.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
                arrival_date += timedelta(days=1)
        
        return {
            'predicted_distance_km': round(predicted_distance, 2),
            'predicted_hours': round(predicted_hours, 2),
            'total_work_hours': total_work_hours,
            'total_work_days': total_workdays,
            'predicted_arrival_date': arrival_date.strftime('%Y-%m-%d'),
            'weekend_adjusted': arrival_date.weekday() == 0 and departure_dt.weekday() < 5
        }

    def print_unique_values(self):
        """Print all unique values from the training data"""
        print("\nValori disponibili nel dataset:")
        print("\nTipi di servizio:", self.unique_values['service_types'])
        print("\nTipi di veicolo:", self.unique_values['vehicle_types'])
        print("\nPaesi di partenza:", self.unique_values['departure_countries'])
        print("\nPaesi di arrivo:", self.unique_values['arrival_countries'])
        print("\nEsempio CAP di partenza:", self.unique_values['departure_zipcodes'][:5])
        print("\nEsempio CAP di arrivo:", self.unique_values['arrival_zipcodes'][:5])

# Esempio di utilizzo
if __name__ == "__main__":
    predictor = DeliveryPredictor()
    
    # Training
    dist_acc, time_acc = predictor.train("delivery_data_pulito.csv")
    print(f"Accuratezza modello distanza: {dist_acc*100:.2f}%")
    print(f"Accuratezza modello tempo: {time_acc*100:.2f}%")
    
    # Commenta o rimuovi la seguente riga se non vuoi stampare i valori disponibili
    # predictor.print_unique_values()
    
    # Example prediction using actual values from the dataset
    prediction = predictor.predict(
        service_type="Corriere espresso",
        vehicle_type="Express",
        departure_country="IT",
        arrival_country="IT",
        departure_zipcode="62010",
        arrival_zipcode="22100",
        departure_date="2023-01-11"
    )
    
    print("\nPrevisione:")
    print(f"Distanza stimata: {prediction['predicted_distance_km']} km")
    print(f"Tempo stimato: {prediction['predicted_hours']} ore")
    print(f"Ore di lavoro totali: {prediction['total_work_hours']} ore")
    print(f"Giorni di lavoro totali: {prediction['total_work_days']} giorni")
    print(f"Data di arrivo prevista: {prediction['predicted_arrival_date']}")
    if prediction['weekend_adjusted']:
        print("Nota: La consegna è stata spostata a lunedì per il weekend")

KeyboardInterrupt: 

In [7]:
#prova Previsione

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from datetime import datetime, timedelta

class DeliveryPredictor:
    def __init__(self):
        self.le_service = LabelEncoder()
        self.le_vehicle = LabelEncoder()
        self.le_dep_country = LabelEncoder()
        self.le_arr_country = LabelEncoder()
        self.le_dep_zip = LabelEncoder()
        self.le_arr_zip = LabelEncoder()
        self.scaler = RobustScaler()  # Cambiato a RobustScaler per gestire meglio gli outlier
        self.distance_model = GradientBoostingRegressor(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        self.time_model = GradientBoostingRegressor(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        
    def prepare_data(self, df):
        # Pulisci e prepara i dati
        df = df.copy()
        
        # Converti i CAP in stringhe e gestisci i valori NaN
        df['DEPARTURE_ZIPCODE'] = df['DEPARTURE_ZIPCODE'].fillna('00000')
        df['ARRIVAL_ZIPCODE'] = df['ARRIVAL_ZIPCODE'].fillna('00000')
        df['DEPARTURE_ZIPCODE'] = df['DEPARTURE_ZIPCODE'].astype(str).str.replace('.0', '').str.zfill(5)
        df['ARRIVAL_ZIPCODE'] = df['ARRIVAL_ZIPCODE'].astype(str).str.replace('.0', '').str.zfill(5)
        
        # Rimuovi outlier dal tempo di consegna
        q1 = df['DELIVERY_TIME_HH'].quantile(0.25)
        q3 = df['DELIVERY_TIME_HH'].quantile(0.75)
        iqr = q3 - q1
        df = df[
            (df['DELIVERY_TIME_HH'] >= q1 - 1.5 * iqr) & 
            (df['DELIVERY_TIME_HH'] <= q3 + 1.5 * iqr)
        ]
        
        # Aggiungi feature derivate
        df['SAME_COUNTRY'] = (df['DEPARTURE_COUNTRY'] == df['ARRIVAL_COUNTRY']).astype(int)
        
        # Encode categorical variables
        df['SERVICETYPE_encoded'] = self.le_service.fit_transform(df['SERVICETYPE'])
        df['VEHICLETYPE_encoded'] = self.le_vehicle.fit_transform(df['VEHICLETYPE'])
        df['DEPARTURE_COUNTRY_encoded'] = self.le_dep_country.fit_transform(df['DEPARTURE_COUNTRY'])
        df['ARRIVAL_COUNTRY_encoded'] = self.le_arr_country.fit_transform(df['ARRIVAL_COUNTRY'])
        df['DEPARTURE_ZIPCODE_encoded'] = self.le_dep_zip.fit_transform(df['DEPARTURE_ZIPCODE'])
        df['ARRIVAL_ZIPCODE_encoded'] = self.le_arr_zip.fit_transform(df['ARRIVAL_ZIPCODE'])
        
        # Store unique values
        self.unique_values = {
            'service_types': df['SERVICETYPE'].unique(),
            'vehicle_types': df['VEHICLETYPE'].unique(),
            'departure_countries': df['DEPARTURE_COUNTRY'].unique(),
            'arrival_countries': df['ARRIVAL_COUNTRY'].unique(),
            'departure_zipcodes': df['DEPARTURE_ZIPCODE'].unique(),
            'arrival_zipcodes': df['ARRIVAL_ZIPCODE'].unique()
        }
        
        # Prepare features
        features = [
            'SERVICETYPE_encoded', 'VEHICLETYPE_encoded', 
            'DEPARTURE_COUNTRY_encoded', 'ARRIVAL_COUNTRY_encoded',
            'DEPARTURE_ZIPCODE_encoded', 'ARRIVAL_ZIPCODE_encoded', 
            'WDAY', 'SAME_COUNTRY'
        ]
        
        # Aggiungi DISTANCE_ENCODED anche se non è presente nel dataset
        if 'DECLARED_DISTANCE_KM' in df.columns:
            df['DISTANCE_ENCODED'] = self.scaler.fit_transform(df[['DECLARED_DISTANCE_KM']])
        else:
            df['DISTANCE_ENCODED'] = 0  # Imposta a 0 se non è presente
        
        features.append('DISTANCE_ENCODED')
        
        X = df[features].values
        X = self.scaler.fit_transform(X)
        
        return X, df

    def train(self, data_path):
        # Load data
        df = pd.read_csv(data_path)
        
        # Prepare features and targets
        X, df_cleaned = self.prepare_data(df)
        y_distance = df_cleaned['DECLARED_DISTANCE_KM'].values
        y_time = df_cleaned['DELIVERY_TIME_HH'].values
        
        # Normalizza il tempo di consegna per ore lavorative (8 ore al giorno)
        y_time = np.minimum(y_time, 72)  # Cap a 72 ore (9 giorni lavorativi)
        
        # Split data
        X_train, X_test, y_dist_train, y_dist_test, y_time_train, y_time_test = train_test_split(
            X, y_distance, y_time, test_size=0.2, random_state=42
        )
        
        # Train models
        self.distance_model.fit(X_train, y_dist_train)
        self.time_model.fit(X_train, y_time_train)
        
        # Calculate accuracy
        dist_accuracy = r2_score(y_dist_test, self.distance_model.predict(X_test))
        time_accuracy = r2_score(y_time_test, self.time_model.predict(X_test))
        
        return dist_accuracy, time_accuracy
    
    def predict(self, service_type, vehicle_type, departure_country, arrival_country,
                departure_zipcode, arrival_zipcode, departure_date):
        # Convert departure_date to weekday (0 = Monday)
        departure_dt = datetime.strptime(departure_date, '%Y-%m-%d')
        wday = departure_dt.weekday()
        
        # Convert zipcodes to strings with leading zeros
        departure_zipcode = str(departure_zipcode).replace('.0', '').zfill(5)
        arrival_zipcode = str(arrival_zipcode).replace('.0', '').zfill(5)
        
        # Calculate same country feature
        same_country = 1 if departure_country == arrival_country else 0
        
        # Prepare input data
        input_data = np.array([[self.le_service.transform([service_type])[0],
                                 self.le_vehicle.transform([vehicle_type])[0],
                                 self.le_dep_country.transform([departure_country])[0],
                                 self.le_arr_country.transform([arrival_country])[0],
                                 self.le_dep_zip.transform([departure_zipcode])[0],
                                 self.le_arr_zip.transform([arrival_zipcode])[0],
                                 wday,
                                 same_country]])
        
        # Scale input data
        input_data_scaled = self.scaler.transform(input_data)
        
        # Predict distance and time
        predicted_distance = self.distance_model.predict(input_data_scaled)[0]
        predicted_time = self.time_model.predict(input_data_scaled)[0]
        
        # Calcola la data di arrivo
        arrival_date = departure_dt + timedelta(hours=predicted_time)
        
        # Adjust arrival date if service type is not E-commerce
        if service_type != "E-commerce":
            if arrival_date.weekday() == 5:  # Saturday
                arrival_date += timedelta(days=2)  # Move to Monday
            elif arrival_date.weekday() == 6:  # Sunday
                arrival_date += timedelta(days=1)  # Move to Monday
        
        return {
            'predicted_distance_km': round(predicted_distance, 2),
            'predicted_hours': round(predicted_time, 2),
            'predicted_arrival_date': arrival_date.strftime('%Y-%m-%d')
        }

# Esempio di utilizzo
if __name__ == "__main__":
    predictor = DeliveryPredictor()
    
    # Training
    dist_acc, time_acc = predictor.train("delivery_data_pulito.csv")
    print(f"Accuratezza modello distanza: {dist_acc*100:.2f}%")
    print(f"Accuratezza modello tempo: {time_acc*100:.2f}%")
    
    # Example prediction using actual values from the dataset
    prediction = predictor.predict(
        service_type="Via gomma",  # Cambia il tipo di servizio se necessario
        vehicle_type="Express",
        departure_country="IT",
        arrival_country="IT",
        departure_zipcode="62010",
        arrival_zipcode="22100",
        departure_date="2023-01-11"
    )
    
    print("\nPrevisione:")
    print(f"Distanza stimata: {prediction['predicted_distance_km']} km")
    print(f"Tempo stimato: {prediction['predicted_hours']} ore")
    print(f"Data di arrivo prevista: {prediction['predicted_arrival_date']}")

Accuratezza modello distanza: 100.00%
Accuratezza modello tempo: 55.96%


ValueError: X has 8 features, but RobustScaler is expecting 9 features as input.